In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score

import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")




In [2]:

with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)

In [3]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

train['dataset_mode'] = 0
valid['dataset_mode'] = 1
test['dataset_mode'] = 2

In [17]:
data = pd.concat([train,valid,test]).reset_index(drop = True)



In [18]:
temp = pd.read_feather('bge_m3_emb_title_feat.pkl')
temp.columns = ['PID','autherID'] + [f'bge_m3_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [19]:
temp = pd.read_feather('feats/bge_m3_abstract_sim_feats.feather')
temp.columns = ['PID','autherID'] + [f'bge_m3_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')


In [20]:
temp = pd.read_feather('feats/abstract_tfidf_sim.feather')
temp = temp.drop(['label','pid_abstract_sim','pid_title_sim', 'pid_title_sim_mean',
       'pid_title_sim_max', 'pid_title_sim_min', 'pid_title_sim_std',
       'pid_title_sim_median'],axis = 1)
temp.columns = ['PID','autherID'] + [f'tfidf_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [21]:
temp = pd.read_feather('feats/title_tfidf_sim.feather')
temp = temp.drop(['label','pid_title_sim'],axis = 1)
temp.columns = ['PID','autherID'] + [f'tfidf_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [22]:
temp = pd.read_feather('feats/venue_tfidf_sim.feather')
temp = temp.drop(['label','pid_venue_sim','pid_title_sim', 'pid_title_sim_mean',
       'pid_title_sim_max', 'pid_title_sim_min', 'pid_title_sim_std',
       'pid_title_sim_median', 'pid_abstract_sim', 'pid_abstract_sim_mean',
       'pid_abstract_sim_max', 'pid_abstract_sim_min', 'pid_abstract_sim_std',
       'pid_abstract_sim_median', 'pid_keyword_text_sim',
       'pid_keyword_text_sim_mean', 'pid_keyword_text_sim_max',
       'pid_keyword_text_sim_min', 'pid_keyword_text_sim_std',
       'pid_keyword_text_sim_median',],axis = 1)
temp.columns = ['PID','autherID'] + [f'tfidf_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [27]:
temp = pd.read_feather('feats/keyword_text_tfidf_sim.feather')
temp = temp.drop(['label','pid_keyword_text_sim', 'pid_title_sim', 'pid_title_sim_mean',
       'pid_title_sim_max', 'pid_title_sim_min', 'pid_title_sim_std',
       'pid_title_sim_median', 'pid_abstract_sim', 'pid_abstract_sim_mean',
       'pid_abstract_sim_max', 'pid_abstract_sim_min', 'pid_abstract_sim_std',
       'pid_abstract_sim_median'],axis = 1)
temp.columns = ['PID','autherID'] + [f'tfidf_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [29]:
temp = pd.read_feather('feats/keywords_jaccard_sim.feather')
temp = temp.drop(['label','pid_sim'],axis = 1)
temp.columns = ['PID','autherID'] + [f'jaccard_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [31]:
temp = pd.read_feather('feats/autherName_jaccard_sim.feather')
temp = temp.drop(['label','pid_sim'],axis = 1)
temp.columns = ['PID','autherID'] + [f'jaccard_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [33]:
temp = pd.read_feather('feats/autherName_raw_jaccard_sim.feather')
temp = temp.drop(['label','pid_sim'],axis = 1)
temp.columns = ['PID','autherID'] + [f'jaccard_{f}' for f in temp.columns[2:]]
data = data.merge(temp,on = ['PID','autherID'],how = 'left')

In [35]:
temp = pd.read_feather('feats/abstract_abstract2vec_feat.feather')
data = data.merge(temp,on = ['PID'],how = 'left')

In [36]:
temp = pd.read_feather('feats/title_tfidf2vec_feat.feather')
data = data.merge(temp,on = ['PID'],how = 'left')

In [37]:
temp = pd.read_feather('feats/venue_venue2vec_feat.feather')
data = data.merge(temp,on = ['PID'],how = 'left')

In [38]:
temp = pd.read_feather('feats/keyword_text_keyword2vec_feat.feather')
data = data.merge(temp,on = ['PID'],how = 'left')

In [39]:
temp = pd.read_feather('feats/w2v_feats/autherID_keyword_w2v_emb.feather')
data=  data.merge(temp,on = 'autherID',how = 'left')

In [40]:
temp = pd.read_feather('feats/w2v_feats/autherID_venue_w2v_emb.feather')
data=  data.merge(temp,on = 'autherID',how = 'left')


#temp = pd.read_feather('feats/w2v_feats/venue_autherID_w2v_emb_mean.feather')
#data=  data.merge(temp,on = ['PID','autherID'],how = 'left')

In [41]:
#title,abstract
t_dict = joblib.load('bge_m3_emb/abstract2vec_dict.pkl')
abstract_emb = pd.DataFrame(t_dict.values()).values
svd_tmp = TruncatedSVD(n_components=16, n_iter=20, random_state=2024)
abstract_emb_svd = svd_tmp.fit_transform(abstract_emb)
abstract_emb_svd = pd.DataFrame(abstract_emb_svd,columns = [f'abstract_emb_svd_{i}' for i in range(16)])
abstract_emb_svd = pd.concat([pd.DataFrame(t_dict.keys(),columns = ['PID']),abstract_emb_svd],axis = 1)
data = data.merge(abstract_emb_svd,on = ['PID'],how = 'left')

In [42]:



t_dict = joblib.load('bge_m3_emb/title2vec_dict.pkl')
abstract_emb = pd.DataFrame(t_dict.values()).values
svd_tmp = TruncatedSVD(n_components=16, n_iter=20, random_state=2024)
abstract_emb_svd = svd_tmp.fit_transform(abstract_emb)
abstract_emb_svd = pd.DataFrame(abstract_emb_svd,columns = [f'title_emb_svd_{i}' for i in range(16)])
abstract_emb_svd = pd.concat([pd.DataFrame(t_dict.keys(),columns = ['PID']),abstract_emb_svd],axis = 1)
data = data.merge(abstract_emb_svd,on = ['PID'],how = 'left')

In [44]:
data['year'] = data['PID'].apply(lambda x:pid[x]['year'])
data['year'] = data['year'].apply(lambda x:int(x) if type(x) == int else np.nan)


In [45]:

df = pd.read_feather('temp_data/autherID_authors.feather')

df['index'] = df.index
df['author_rank'] = df.groupby(['autherID','PID'])['index'].rank()

temp = (df[df['flag'] == 1].groupby(['autherID','PID'])['author_rank'].mean()  / df.groupby(['autherID','PID'])['author_rank'].max()).reset_index()

temp.columns = ['autherID','PID','author_rank_ratio']
data = data.merge(temp,on = ['autherID','PID'],how = 'left')
data = data.merge(df[df['flag'] == 1][['autherID','PID','author_rank']].groupby(['autherID','PID'])['author_rank'].mean().reset_index(),on = ['autherID','PID'],how = 'left')

In [48]:
temp = pd.read_feather('feats/abstract_w2v_sim.feather')
temp = temp.drop(['label','abstract_w2v_sim'],axis = 1)
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

In [49]:
temp = pd.read_feather('feats/title_w2v_sim.feather')
temp = temp.drop(['label','title_w2v_sim'],axis = 1)
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

In [52]:
temp = pd.read_feather('chatglm_pid_title_sim.pkl')
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

In [54]:
temp = pd.read_feather('chatglm_pid_abstract_sim.pkl')
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

In [56]:
temp = pd.read_feather('chatglm_pid_title_svd.pkl')
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

temp = pd.read_feather('chatglm_pid_abstract_svd.pkl')
data = data.merge(temp ,on = ['autherID','PID'],how = 'left')

In [59]:
data.to_feather('all_feat.feather')

In [60]:


df_train = data[data['label'].notna()].reset_index(drop = True)
df_test = data[data['label'].isna()].reset_index(drop = True)
df_train['label'] = 1 - df_train['label']

drop_feat =['tfidf_autherName','dataset_mode','id'] + ['PID','label','autherID','pid_title','autherName', 'abstract', 'keywords', 'venue', 'authors', 'index','cos_sim_pid','autherName_count'] 
used_feat = [f for f in df_train.columns if f not in (['PID','label','autherID'] + drop_feat )]
print(len(used_feat))
print(used_feat)
ycol = 'label'

#del data
gc.collect()

372
['bge_m3_pid_title_sim_mean', 'bge_m3_pid_title_sim_max', 'bge_m3_pid_title_sim_min', 'bge_m3_pid_title_sim_std', 'bge_m3_pid_title_sim_median', 'bge_m3_pid_abstract_sim_mean', 'bge_m3_pid_abstract_sim_max', 'bge_m3_pid_abstract_sim_min', 'bge_m3_pid_abstract_sim_std', 'bge_m3_pid_abstract_sim_median', 'tfidf_pid_abstract_sim_mean', 'tfidf_pid_abstract_sim_max', 'tfidf_pid_abstract_sim_min', 'tfidf_pid_abstract_sim_std', 'tfidf_pid_abstract_sim_median', 'tfidf_pid_title_sim_mean', 'tfidf_pid_title_sim_max', 'tfidf_pid_title_sim_min', 'tfidf_pid_title_sim_std', 'tfidf_pid_title_sim_median', 'tfidf_pid_venue_sim_mean', 'tfidf_pid_venue_sim_max', 'tfidf_pid_venue_sim_min', 'tfidf_pid_venue_sim_std', 'tfidf_pid_venue_sim_median', 'tfidf_pid_keyword_text_sim_mean', 'tfidf_pid_keyword_text_sim_max', 'tfidf_pid_keyword_text_sim_min', 'tfidf_pid_keyword_text_sim_std', 'tfidf_pid_keyword_text_sim_median', 'jaccard_pid_keyword_cnt_1_mean', 'jaccard_pid_keyword_cnt_1_max', 'jaccard_pid_keywor

0